In [1]:
__PRODUCTION__ = 1
__NAME__       = 'er-analysis'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.55

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure
import matplotlib.pyplot as plt

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import functools
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import networkx as nx
import pickle
from sklearn import metrics

from symfac.experimental import RBFExpansionV2

In [3]:
svd2_approx, rbf2_approx,K00=pickle.load(open('data/paper/er_approx_2.pickle', "rb"))
fpr2, tpr2, fpr_svd2, tpr_svd2 = pickle.load(open('data/paper/er_auc_2.pickle', "rb"))
fpr7, tpr7, fpr_svd7, tpr_svd7 = pickle.load(open('data/paper/er_auc_7.pickle', "rb"))

In [4]:
fig = SuperFigure(plt.figure(figsize=(__WIDTH__, __HEIGHT__), dpi=300))
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
# x0 = 0.05
# dx = 0.24
# w = 0.21
x0 = 0.005
dx = 0.245
w = 0.23
axs = [
    fig.make_axes(
        left=x0 + dx * i,
        width=w,
        top=0.09,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]
bbox = axs[3].get_position()
bbox.x0 += 0.02
bbox.x1 += 0.02
axs[3].set_position(bbox)

image_style = dict(
    vmin=0,
    vmax=1,
    cmap='Greys_r'
)
axs[0].imshow(K00, **image_style)
axs[1].imshow(rbf2_approx, **image_style)
axs[2].imshow(svd2_approx, **image_style)

svd_base_style = dict(
    lw=0.75,
    dash_capstyle='round'
)
svd2_style = dict(
    ls=(2, (1.25, 2.5)),
    **svd_base_style
)
svd7_style = dict(
    ls=(4, (0.5, 2.5, 3, 2.5)),
    **svd_base_style
)
rbf_base_style = dict(
    lw=1.0,
    markersize=3,
    markeredgewidth=0.75,
    markerfacecolor='w',
)
rbf2_style = dict(
    marker='o',
    markevery=30,
    **rbf_base_style
)
rbf7_style = dict(
    marker='^',
    markevery=5,
    **rbf_base_style
)
lo_style = dict(
    color='#F06060'
)
hi_style = dict(
    color='#606060'
)
axs[3].plot(fpr_svd2, tpr_svd2, label='SVD (2), AUC = 0.72',  **svd7_style, **lo_style)
axs[3].plot(fpr2, tpr2, label='RBF (2), AUC = 0.82', **rbf2_style, **lo_style)
axs[3].plot(fpr_svd7, tpr_svd7, label='SVD (7), AUC = 0.90',  **svd2_style, **hi_style)
axs[3].plot(fpr7, tpr7, label='RBF (7), AUC = 0.98',  **rbf7_style, **hi_style)

title_style = dict(
    fontsize=7,
    y=1.02,
    va='top'
)
tick_style = dict(
    fontsize=7,
)
label_style = dict(
    fontsize=8,
    labelpad=0.75,
)


for i in range(4):
    if i != 3:
        ticks = [0, 40]
        axs[i].set_xbound(lower=0, upper=40)
        axs[i].set_ybound(lower=0, upper=40)
    else:
        ticks = [0, 1]
        axs[i].set_xbound(lower=-0.02, upper=1.02)
        axs[i].set_ybound(lower=-0.02, upper=1.02)
    axs[i].set_xticks(ticks)
    axs[i].set_yticks(ticks)
    axs[i].set_xticklabels(['%d' % x for x in ticks], **tick_style)
    if i == 3:
        axs[i].set_yticklabels(['%d' % y for y in ticks], **tick_style)
    else:
        axs[i].set_yticklabels([])

axs[0].set_title(r'\textbf{Adjacency matrix}', **title_style)
axs[1].set_title(r'\textbf{2-Component RBF}', **title_style)
axs[2].set_title(r'\textbf{2-Rank SVD}', **title_style)
axs[3].set_title(r'\textbf{AUC-ROC}', **title_style)
axs[3].legend(
    loc='lower right',
    fontsize=7,
    frameon=False
)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

In [5]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-er-analysis.pdf


# END